# Exporting to BIDS Formats


The [Brain Imaging Data Structure (BIDS)](https://bids.neuroimaging.io/index.html) is a comprehensive framework designed to systematically organize and share diverse types of data, including behavioral, physiological, and neuroimaging information. Converting datasets into BIDS format is a widely adopted methodology, particularly in the process of curating datasets that adhere to the principles of FAIR (Findable, Accessible, Interoperable, Reusable).

For datasets encompassing mobile eye-tracking data, it is essential to apply specific BIDS specifications tailored for such data. In this context, Motion-BIDS and Eye-Tracking-BIDS specifications are noteworthy. Motion-BIDS ([BEP029](https://github.com/bids-standard/bids-specification/pull/981)) has been successfully integrated into the official BIDS specification, demonstrating its readiness for use in organizing motion-related data. On the other hand, Eye-Tracking-BIDS ([BEP020](https://github.com/bids-standard/bids-specification/pull/1128)) is still undergoing development, reflecting ongoing efforts to provide a standardized format for eye-tracking data. You can find more information about these specifications in the following references:

> <cite>Gorgolewski, K., Auer, T., Calhoun, V. et al. The brain imaging data structure, a format for organizing and describing outputs of neuroimaging experiments. Sci Data 3, 160044 (2016). https://doi.org/10.1038/sdata.2016.44<cite>

> <cite>Jeung, S., Cockx, H., Appelhoff, S. et al. Motion-BIDS: an extension to the brain imaging data structure to organize motion data for reproducible research. Sci Data 11, 716 (2024). https://doi.org/10.1038/s41597-024-03559-8<cite>

In the ensuing section, we will delve into the procedure for exporting data to Motion-BIDS. This will be accomplished using the `export_motion_bids` method available within PyNeon's `Recording` objects, offering a practical guide for researchers aiming to standardize their motion data in alignment with the BIDS framework.

In [1]:
import json
from pathlib import Path
import pandas as pd
from pyneon import Dataset, get_sample_data

dataset = Dataset(get_sample_data("markers", format="cloud"))
rec = dataset.recordings[0]

To use `export_motion_bids` method, we need to specify the output directory where the BIDS dataset will be saved, and a string prefix to denote this session of data. The prefix follows the following format (Fields in [] are optional):

```text
sub-<label>[_ses-<label>]_task-<label>_tracksys-<label>[_acq-<label>][_run-<index>]
```

If you have any additional metadata that you would like to include, you can pass it as a dictionary to the `extra_metadata` argument. This metadata will be saved in the `dataset_description.json` file.

Let's see what files will be exported to the BIDS dataset directory:

In [2]:
# Create a BIDS directory
motion_dir = Path("export") / "BIDS" / "sub-1" / "motion"
motion_dir.mkdir(exist_ok=True, parents=True)

# Export the motion data to BIDS format
prefix = "sub-1_task-screenFlash_tracksys-NeonIMU_run-1"
extra_metadata = {
    "TaskName": "screenFlash",
    "InstitutionName": "Streeling University",
    "InstitutionAddress": "Trantor, Galactic Empire",
    "InstitutionalDepartmentName": "Department of Psychohistory",
}

rec.export_motion_bids(motion_dir, prefix=prefix, extra_metadata=extra_metadata)

# Print all the conents of motion_dir
for path in motion_dir.iterdir():
    print(path.name)

sub-1_task-screenFlash_tracksys-NeonIMU_run-1_channels.json
sub-1_task-screenFlash_tracksys-NeonIMU_run-1_channels.tsv
sub-1_task-screenFlash_tracksys-NeonIMU_run-1_motion.json
sub-1_task-screenFlash_tracksys-NeonIMU_run-1_motion.tsv


The contents of these files follow the Motion-BIDS specification at: https://bids-specification.readthedocs.io/en/stable/modality-specific-files/motion.html.

For example, the `_motion.tsv` is a tab-separated values file that contains the (n_samples, n_channels) motion data without a header:

In [3]:
motion_tsv_path = motion_dir / f"{prefix}_motion.tsv"
motion_df = pd.read_csv(motion_tsv_path, sep="\t")
print(f"Motion data shape: {motion_df.shape}")
print(motion_df.head())

Motion data shape: (26555, 13)
   -51.717758  -2.668381  60.407639  -0.239258  -0.118164  0.899902  \
0  -57.800365  -2.150263  66.800467  -0.232189  -0.139369  0.913207   
1  -63.838428  -1.759561  73.407542  -0.214823  -0.143753  0.923766   
2  -72.211448  -0.018978  84.477888  -0.208008  -0.153056  0.926946   
3  -77.977801   2.357307  92.820179  -0.196248  -0.166075  0.932501   
4  -80.598493   4.727259  97.355536  -0.168772  -0.180748  0.941449   

   7.143753693332938  -3.1793275574051094  -106.20521917582651  \
0           7.205822            -3.592368          -105.606506   
1           7.273488            -4.038350          -104.953976   
2           7.357109            -4.546941          -104.196205   
3           7.462461            -5.104590          -103.347745   
4           7.587478            -5.685176          -102.441670   

   0.5976047515869141  0.0331803187727928  0.0595318600535392  \
0            0.601491            0.031118            0.062889   
1            0.

Its metadata is stored in the `_motion.json` file, which contains (note the extra metadata we added):

In [4]:
motion_json = motion_dir / f"{prefix}_motion.json"
with open(motion_json, "r") as f:
    motion_json_data = json.load(f)
print(json.dumps(motion_json_data, indent=2))

{
  "TaskName": "screenFlash",
  "TaskDescription": "",
  "Instructions": "",
  "DeviceSerialNumber": "114837",
  "Manufacturer": "TDK InvenSense & Pupil Labs",
  "ManufacturersModelName": "ICM-20948",
  "SoftwareVersions": "App version: 2.9.26-prod; Pipeline version: 2.8.0",
  "InstitutionName": "Streeling University",
  "InstitutionAddress": "Trantor, Galactic Empire",
  "InstitutionalDepartmentName": "Department of Psychohistory",
  "SamplingFrequency": 110,
  "ACCELChannelCount": 3,
  "GYROChannelCount": 3,
  "MissingValues": "n/a",
  "MotionChannelCount": 13,
  "ORNTChannelCount": 7,
  "SubjectArtefactDescription": "",
  "TrackedPointsCount": 0,
  "TrackingSystemName": "IMU included in Neon"
}


The metadata for each channel (each column in the `_motion.tsv` file) is stored in `_channels.tsv` file:

In [5]:
channels_tsv_path = motion_dir / f"{prefix}_channels.tsv"
channels_df = pd.read_csv(channels_tsv_path, sep="\t")
print(channels_df)

              name component   type tracked_point      units  \
0           gyro x         x   GYRO          Head      deg/s   
1           gyro y         y   GYRO          Head      deg/s   
2           gyro z         z   GYRO          Head      deg/s   
3   acceleration x         x  ACCEL          Head          g   
4   acceleration y         y  ACCEL          Head          g   
5   acceleration z         z  ACCEL          Head          g   
6             roll         x   ORNT          Head        deg   
7            pitch         y   ORNT          Head        deg   
8              yaw         z   ORNT          Head        deg   
9     quaternion w         w   ORNT          Head  arbitrary   
10    quaternion x         x   ORNT          Head  arbitrary   
11    quaternion y         y   ORNT          Head  arbitrary   
12    quaternion z         z   ORNT          Head  arbitrary   

    sampling_frequency  
0                  110  
1                  110  
2                  110  
3  

Finally, the `_channels.json` file contains the coordinate system:

In [6]:
channels_json_path = motion_dir / f"{prefix}_channels.json"
with open(channels_json_path, "r") as f:
    channels_json_data = json.load(f)
print(json.dumps(channels_json_data, indent=2))

{
  "reference_frame": {
    "Levels": {
      "global": {
        "SpatialAxes": "RAS",
        "RotationOrder": "ZXY",
        "RotationRule": "right-hand",
        "Description": "This global reference frame is defined by the IMU axes: X right, Y anterior, Z superior. The scene camera frame differs from this frame by a 102-degree rotation around the X-axis. All motion data are expressed relative to the IMU frame for consistency."
      }
    }
  }
}
